In [ ]:
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN
from tensorflow.keras.models import load_model
from tensorflow import keras

def scaling(x, scale=1):
    return x * scale

keras.utils.get_custom_objects()["Custom>scaling"] = scaling
embedder = load_model("./facenet_finetuned.h5")
detector = MTCNN()

def get_embedding(image):
    results = detector.detect_faces(image)
    if len(results) == 0:
        return None, None

    x, y, width, height = results[0]["box"]
    face_crop = image[y : y + height, x : x + width]
    face_resized = cv2.resize(face_crop, (160, 160))
    face_resized = face_resized.astype("float32")
    face_resized = np.expand_dims(face_resized, axis=0)

    embedding = embedder.predict(face_resized)[0]
    return embedding, (x, y, width, height)

reference_img_path = "" # INSERT PICTURE PATH HERE
reference_bgr = cv2.imread(reference_img_path)
reference_rgb = cv2.cvtColor(reference_bgr, cv2.COLOR_BGR2RGB)

reference_embedding, _ = get_embedding(reference_rgb)
if reference_embedding is None:
    print("No face detected in reference image")
    exit()

cap = cv2.VideoCapture(0)
print("Press 'q' to quit…")

while True:
    ret, frame_bgr = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
    live_embedding, bbox = get_embedding(frame_rgb)
    if live_embedding is not None:
        dist = np.linalg.norm(reference_embedding - live_embedding)
        x, y, w, h = bbox
        color = (0, 255, 0) if dist < 0.9 else (0, 0, 255)
        cv2.rectangle(frame_bgr, (x, y), (x + w, y + h), color, 2)
        cv2.putText(
            frame_bgr,
            f"Distance: {dist:.2f}",
            (x, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            color,
            2,
        )

    cv2.imshow("Live Feed", frame_bgr)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
